Goal: use Panadas Dataframe Merge to join many related tables into a single table

In [ ]:
import arcpy
#import time
import pandas as pd

In [ ]:
# Poles use shape@ to create point feature for damage assessment
PoleFldsOrig = ['GLOBALID','SUBTYPE','OWNER','FACILITYID','POLESIZE','MATERIAL','COLOR','TREATMENTTYPE','YEARMANUFACTURED','STUBDATE','BILLINGGROUP','FEEDERIDS','INSTALL_NUM','UNIT_TYPE','STREETADDRESS','BACKBONEINDICATOR','StockNumber','SHAPE@']
PoleFldsDest = ['eSupportStructure_GLOBALID','eSupportStructure_SUBTYPE','eSupportStructure_OWNER','eSupportStructure_FACILITYID','eSupportStructure_POLESIZE','eSupportStructure_MATERIAL','eSupportStructure_COLOR','eSupportStructure_TREATMENTTYPE','eSupportStructure_YEARMANUFACTURED','eSupportStructure_STUBDATE','eSupportStructure_BILLINGGROUP','eSupportStructure_FEEDERIDS','eSupportStructure_INSTALLNUM','eSupportStructure_UNITTYPE','eSupportStructure_STREETADDRESS','eSupportStructure_BACKBONE','eSupportStructure_StockNumber','SHAPE@']
PoleFldsDestOut = ['eSupportStructure_GLOBALID','eSupportStructure_SUBTYPE','eSupportStructure_OWNER','eSupportStructure_FACILITYID','eSupportStructure_POLESIZE','eSupportStructure_MATERIAL','eSupportStructure_COLOR','eSupportStructure_TREATMENTTYPE','eSupportStructure_YEARMANUFACTURED','eSupportStructure_STUBDATE','eSupportStructure_BILLINGGROUP','eSupportStructure_FEEDERIDS','eSupportStructure_INSTALLNUM','eSupportStructure_UNITTYPE','eSupportStructure_STREETADDRESS','eSupportStructure_BACKBONE','eSupportStructure_StockNumber']

# feature classes that have 1:1 relationship with poles
CapBankFldsOrig = ['eSupportStructure_GLOBALID','DEVICEID','SUBTYPE','STRUCTUREID','GLOBALID']
CapBankFldsRename = {'DEVICEID':'eCapacitorBank_DEVICEID','SUBTYPE':'eCapacitorBank_SUBTYPE','STRUCTUREID':'eCapacitorBank_STRUCTUREID','GLOBALID':'eCapacitorBank_GLOBALID'}
CapBankFldsDest = ['eSupportStructure_GLOBALID','eCapacitorBank_DEVICEID','eCapacitorBank_SUBTYPE','eCapacitorBank_STRUCTUREID','eCapacitorBank_GLOBALID']

RecloserFldsOrig = ['eSupportStructure_GLOBALID','DEVICEID','FEEDERID','FEEDERID2','PHASEDESIGNATION','SUBTYPE','AMPRATING','BACKBONEPASSTHRU','CUSTOMERCOUNT','STRUCTUREID','INTERRUPTINGTYPE','INTERRUPTRATING','CONTROLLERTYPE','BACKBONEINDICATOR','GLOBALID']
RecloserFldsRename = {'DEVICEID':'eRecloser_DEVICEID','FEEDERID':'eRecloser_FEEDERID','FEEDERID2':'eRecloser_FEEDERID2','PHASEDESIGNATION':'eRecloser_PHASE','SUBTYPE':'eRecloser_SUBTYPE','AMPRATING':'eRecloser_AMPRATING','BACKBONEPASSTHRU':'eRecloser_BACKBONEPASSTHRU','CUSTOMERCOUNT':'eRecloser_CUSTOMERCOUNT','STRUCTUREID':'eRecloser_STRUCTUREID','INTERRUPTINGTYPE':'eRecloser_INTERRUPTINGTYPE','INTERRUPTRATING':'eRecloser_INTERRUPTRATING','CONTROLLERTYPE':'eRecloser_CONTROLLERTYPE','BACKBONEINDICATOR':'eRecloser_BACKBONEINDICATOR','GLOBALID':'eRecloser_GLOBALID'}
RecloserFldsDest = ['eSupportStructure_GLOBALID','eRecloser_DEVICEID','eRecloser_FEEDERID','eRecloser_FEEDERID2','eRecloser_PHASE','eRecloser_SUBTYPE','eRecloser_AMPRATING','eRecloser_BACKBONEPASSTHRU','eRecloser_CUSTOMERCOUNT','eRecloser_STRUCTUREID','eRecloser_INTERRUPTINGTYPE','eRecloser_INTERRUPTRATING','eRecloser_CONTROLLERTYPE','eRecloser_BACKBONE','eRecloser_GLOBALID']

SectionalizerFldsOrig = ['eSupportStructure_GLOBALID','DEVICEID','FEEDERID','FEEDERID2','BACKBONEPASSTHRU','CUSTOMERCOUNT','STRUCTUREID','GLOBALID']
SectionalizerFldsRename = {'DEVICEID':'eSectionalizer_DEVICEID','FEEDERID':'eSectionalizer_FEEDERID','FEEDERID2':'eSectionalizer_FEEDERID2','BACKBONEPASSTHRU':'eSectionalize_BACKBONEPASSTHRU','CUSTOMERCOUNT':'eSectionalizer_CUSTOMERCOUNT','STRUCTUREID':'eSectionalizer_STRUCTUREID','GLOBALID':'eSectionalizer_GLOBALID'}
SectionalizerFldsDest = ['eSupportStructure_GLOBALID','eSectionalizer_DEVICEID','eSectionalizer_FEEDERID','eSectionalizer_FEEDERID2','eSectionalize_BACKBONEPASSTHRU','eSectionalizer_CUSTOMERCOUNT','eSectionalizer_STRUCTUREID','eSectionalizer_GLOBALID']

VoltRegBankFldsOrig = ['eSupportStructure_GLOBALID','DEVICEID','FEEDERID','STRUCTUREID','GLOBALID']
VoltRegBankFldsDest = ['eSupportStructure_GLOBALID','eVoltRegBank_DEVICEID','eVoltRegBank_FEEDERID','eVoltRegBank_STRUCTUREID','eVoltRegBank_GLOBALID']


In [ ]:
# fGDB variables
unitTable = r'C:\Users\friendde\Documents\ArcGIS\Projects\PandasDataframeMerge\PandasDataframeMerge.gdb\MIMS\mmPoleEquipment'
connectionRoot = r'C:\Users\friendde\Documents\ArcGIS\Projects\PandasDataframeMerge\PandasDataframeMerge.gdb\%s'
workspace = r'C:\Users\friendde\Documents\ArcGIS\Projects\PandasDataframeMerge\PandasDataframeMerge.gdb'
origFGDB = r'C:\GISData\Data\Snapshot\mxElectric.geodatabase\main.'

# feature classes
MM_POLEEQUIPMENT_XML = r"C:\Users\friendde\Documents\ArcGIS\Projects\PandasDataframeMerge\MIMS_DATASET_20181019.XML"
poleFC = r'C:\GISData\Data\Snapshot\mxElectric.geodatabase\Electric\main.eSupportStructure'
origFC = ['eCapacitorBank','eRecloser','eSectionalizer','eVoltageRegulatorBank']
fuseFC = 'eFuse'
lightFC = 'eLight'
#poleFC = 'eSupportStructure'
servicepointFC = 'eServicePoint'
switchFC = 'eSwitch'
transformerFC = 'eTransformerBank'

In [ ]:
# prep unitTable
if not arcpy.Exists(unitTable):
    arcpy.ImportXMLWorkspaceDocument_management(workspace, MM_POLEEQUIPMENT_XML, "Schema_Only")
else:
    print('Truncating...',unitTable)
    arcpy.TruncateTable_management(unitTable)

In [ ]:
# Start Main
with arcpy.da.Editor(workspace) as edit:
    print('Inserting...poles')
    ic = arcpy.da.InsertCursor(unitTable,PoleFldsDest)
    with arcpy.da.SearchCursor(poleFC,PoleFldsOrig) as sc:
        for scrow in sc:
            ic.insertRow(scrow)
    del ic
    del sc

In [ ]:
nparrUnitTable = arcpy.da.TableToNumPyArray(unitTable,PoleFldsDestOut,skip_nulls=True)
dfUnitTable = pd.DataFrame(nparrUnitTable)
#pdarr.to_csv(custAcctFile,header=False, index=False)
dfUnitTable

In [ ]:
nparrCapBank = arcpy.da.TableToNumPyArray(r'C:\GISData\Data\Snapshot\mxElectric.geodatabase\main.eCapacitorBank',CapBankFldsOrig,skip_nulls=True)
dfCapBank = pd.DataFrame(nparrCapBank)
#pdarr.to_csv(custAcctFile,header=False, index=False)
dfCapBank

DataFrame.merge(right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html

In [ ]:
df2 = pd.merge(dfUnitTable,dfCapBank, how='outer', on='eSupportStructure_GLOBALID', left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=False, indicator=False, validate=None)

In [ ]:
df2

In [ ]:
df2.rename(index=str,inplace=True,columns=CapBankFldsRename)

In [ ]:
df2

In [ ]:
nparrRecloser = arcpy.da.TableToNumPyArray(r'C:\GISData\Data\Snapshot\mxElectric.geodatabase\main.eRecloser',RecloserFldsOrig,skip_nulls=True)
dfRecloser = pd.DataFrame(nparrRecloser)
#pdarr.to_csv(custAcctFile,header=False, index=False)
dfRecloser

In [ ]:
dfRecloser.rename(index=str,inplace=True,columns=RecloserFldsRename)

In [ ]:
dfRecloser.head()

In [ ]:
df3 = pd.merge(df2, dfRecloser, how='outer', on='eSupportStructure_GLOBALID', left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=False, indicator=False, validate=None)


In [ ]:
list(df3)

In [ ]:
nparrSectionalizer = arcpy.da.TableToNumPyArray(r'C:\GISData\Data\Snapshot\mxElectric.geodatabase\main.eSectionalizer',SectionalizerFldsOrig,skip_nulls=True)
dfSectionalizer = pd.DataFrame(nparrSectionalizer)
dfSectionalizer

In [ ]:
dfSectionalizer.rename(index=str,inplace=True,columns=SectionalizerFldsRename)

In [ ]:
df4 = pd.merge(df3, dfSectionalizer, how='outer', on='eSupportStructure_GLOBALID', left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=False, indicator=False, validate=None)

In [ ]:
list(df4)